Data fields
Each row of the training data contains a click record, with the following features.

ip: ip address of click.
app: app id for marketing.
device: device type id of user mobile phone (e.g., iphone 6 plus, iphone 7, huawei mate 7, etc.)
os: os version id of user mobile phone
channel: channel id of mobile ad publisher
click_time: timestamp of click (UTC)
attributed_time: if user download the app for after clicking an ad, this is the time of the app download
is_attributed: the target that is to be predicted, indicating the app was downloaded
Note that ip, app, device, os, and channel are encoded.

The test data is similar, with the following differences:

click_id: reference for making predictions
is_attributed: not included

Soruce: https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/data

In [9]:
# Python 2 & 3 Compatibility
from __future__ import print_function, division

# Necessary imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.cross_validation import train_test_split
from sklearn import svm


import os
import pickle
%matplotlib inline
from datetime import datetime
from dateutil.parser import parse

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
with open("all_converted_data.pkl", 'rb') as picklefile: 
    df_new = pickle.load(picklefile)
df_new.shape


(1370538, 13)

In [11]:
df_new.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,hour,date,day,month,year
0,117867,12,1,18,19,2017-11-08 23:43:20,NaN,0,23,2017-11-08,8,11,2017
1,43715,17,1,13,280,2017-11-08 05:36:19,NaN,0,5,2017-11-08,8,11,2017
2,183537,2,1,13,477,2017-11-06 17:21:36,NaN,0,17,2017-11-06,6,11,2017
3,46323,3,1,13,402,2017-11-09 06:26:40,NaN,0,6,2017-11-09,9,11,2017
4,4248,9,1,13,489,2017-11-07 10:28:31,NaN,0,10,2017-11-07,7,11,2017


In [12]:
df_download = df_new[df_new.is_attributed == 1].sample(5000)
df_notDownload = df_new[df_new.is_attributed != 1].sample(5000*2)
df_all = df_notDownload.append(df_download, ignore_index=True)

In [13]:
df_all.shape

(15000, 13)

In [14]:
from sqlalchemy import create_engine
import pandas as pd

In [15]:
# create your own connection, use '\list' in psql to find the name and owner of the database
cnx = create_engine('HERE-')

In [16]:
pd.read_sql_query('''SELECT * FROM activity LIMIT 5''',cnx)

,id,ip,app,device,os,channel,click_time,attributed_time,is_attributed


In [77]:
df_sql = df_all.iloc[:,:8]
# df_sql = df_sql.loc[:,['ip','app','device','os','channel', 'is_attributed']]
df_sql.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,104271,12,1,19,219,2017-11-09 14:16:58,NaN,0
1,119524,3,1,10,137,2017-11-09 09:29:47,NaN,0
2,172914,12,1,13,265,2017-11-08 15:46:28,NaN,0
3,60924,15,1,9,130,2017-11-08 03:13:49,NaN,0
4,158596,18,3032,607,107,2017-11-06 18:03:58,NaN,0


In [82]:
len(df_sql.columns)

8

In [83]:
import io

import pandas as pd
from sqlalchemy import create_engine

def write_to_table(df, cnx, schema, table_name, file_name):
    df.to_csv(file_name, sep=',', index=False)
    with open(file_name, 'r') as f: 
        conn = cnx.raw_connection()
        cursor = conn.cursor()
        cmd = "COPY %s(%s) FROM STDIN DELIMITER ',' CSV HEADER; " % (table_name, schema )
        print(cmd)
        cursor.copy_expert(cmd, f)
        conn.commit()

# FROM https://stackoverflow.com/questions/31997859/bulk-insert-a-pandas-dataframe-using-sqlalchemy?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa

In [85]:
write_to_table(df_sql, cnx,'ip,app,device,os,channel,click_time,attributed_time, is_attributed','activity', 'activity_sql.csv' )

COPY activity(ip,app,device,os,channel,click_time,attributed_time, is_attributed) FROM STDIN DELIMITER ',' CSV HEADER; 


In [91]:
pd.read_sql_query('''SELECT AVG(is_attributed) AS AVG_is_attributed, COUNT(*), AVG(device) AS avg_device FROM activity GROUP BY app''',cnx)

,avg_is_attributed,count,avg_device
0,1.000000,1,1.000000
1,1.000000,9,165.888889
2,1.000000,1,1.000000
3,0.238636,264,1.284091
4,0.000000,1,1.000000
5,0.944444,18,1.000000
6,1.000000,8,1.000000
7,0.000000,1,3032.000000
8,0.000000,5,1.000000
9,1.000000,13,1.230769
